In [6]:
SELECT *
FROM szallashely

(21 rows affected)

Total execution time: 00:00:00.044

SZALLAS_ID,SZALLAS_NEV,HELY,CSILLAGOK_SZAMA,TIPUS,ROGZITETTE,ROGZ_IDO,CIM
1,Sába-Ház,Balaton-dél,0,vendégház,Béla,2016-02-28,"8630 Balatonboglár, Radnóti Miklós utca 8"
2,Családi Ház,Balaton-dél,0,vendégház,Béla,2016-03-02,"8630 Balatonboglár, József Attila utca 25"
3,Fortuna Apartman,Hajdúbihar megye,0,Apartman,Sára,2016-03-06,"4200 Hajdúszoboszló, Wesselényi utca 56."
4,Fortuna panzió,Budapest,3,panzió,Béla,2016-03-07,1019 Budapest XIV. Cinkotai út 86.
5,Fortuna Panzió,Békés megye,3,panzió,Sára,2016-03-08,"5900 Orosháza, Gyopárosfürdő Tópart utca 3."
6,Kentaur Hotel,Balaton-dél,3,Hotel,Béla,2016-03-08,"8600 Siófok (Széplak), Akácfa utca 1"
7,Szieszta Apartmanház,Balaton-dél,0,Apartman,Mari,2016-03-11,"8630 Balatonboglár, Dózsa György utca 108"
8,Hotel Három Hattyú,Balaton-dél,4,Hotel,Béla,2016-03-13,"8623 Balatonföldvár, Rákóczi Ferenc út 45."
9,Jáde panzió,Balaton-dél,3,panzió,Mari,2016-03-16,"8624 Balatonföldvár, Kiss u.2."
10,Lagúna Hotel,Budapest,4,Hotel,Sára,2016-03-20,"1097 Budapest, Albert Flórián út 3"


1\. Készítsünk listát az egyes szálláshelyek átlagos csillagszámáról típus, illetve hely szerint csoportosítva (külön-külön)!

- Az átlag 2 tizedesre kerekítve jelenjen meg
- Hagyjuk ki azokat a szálláshelyeket, amelyek nevében kötőjel (-) van!

In [8]:
SELECT SZALLAS_NEV, AVG(CSILLAGOK_SZAMA) AS 'Csillagok számának átlaga'
FROM szallashely
WHERE SZALLAS_NEV NOT LIKE '%-'
GROUP BY GROUPING SETS((TIPUS), (HELY), (SZALLAS_NEV))

(32 rows affected)

Total execution time: 00:00:00.026

SZALLAS_NEV,Csillagok számának átlaga
Bagoly Hotel,3
Családi Ház,0
Fortuna Apartman,0
Fortuna panzió,3
Gold Hotel,3
Hotel Három Hattyú,4
Jáde panzió,3
Kentaur Hotel,3
Kollégium,0
Lagúna Hotel,4


2\. Rangsoroljuk az egyes szobákat a foglalások száma, azon belül férőhely szerint!

- Holtverseny esetén a sorszámok sűrűn kövessék egymást
- Az 1-es számot a legtöbbet foglalt, azon belül legkevesebb férőhelyű szoba kapja
- Csak a szoba azonosítója, a férőhely és a helyezés jelenjen meg

In [25]:
SELECT SZOBA_ID, FEROHELY, DENSE_RANK() OVER (PARTITION BY COUNT(SZOBA_ID) ORDER BY FEROHELY ASC) AS 'Rangsor'
FROM Vendeg v LEFT JOIN Foglalas f ON NEV = f.UGYFEL_FK
              LEFT JOIN Szoba sz ON f.SZOBA_FK = sz.SZOBA_ID
              LEFT JOIN Szallashely szh ON sz.SZALLAS_FK = szh.SZALLAS_ID
GROUP BY SZOBA_ID, FEROHELY
ORDER BY 'Rangsor'

Warning: Null value is eliminated by an aggregate or other SET operation.

(1 row affected)

Total execution time: 00:00:00.013

SZOBA_ID,FEROHELY,Rangsor
NULL,NULL,1


3\. Kérdezzük le az átlagosnál kevesebb főre történt foglalások adatait!

a. Egy új oszlopban jelenjen meg az ügyfél neve is!

b. Egy másik új oszlop mutassa meg, hogy az ügyfél előző három foglalása átlagosan hány főre történt! Az oszlop neve legyen 'Előző három foglalás átlaga'

c. A listában azon vendégek is jelenjenek meg, akik még nem foglaltak!

In [28]:
SELECT  NEV,
        AVG(FELNOTT_SZAM+GYERMEK_SZAM) AS 'Előző három foglalás átlaga'
FROM Vendeg v LEFT JOIN Foglalas f ON NEV = f.UGYFEL_FK
              LEFT JOIN Szoba sz ON f.SZOBA_FK = sz.SZOBA_ID
              LEFT JOIN Szallashely szh ON sz.SZALLAS_FK = szh.SZALLAS_ID
WHERE AVG(FELNOTT_SZAM+GYERMEK_SZAM) < ALL(
    SELECT NEV
    FROM szallashely

)
GROUP BY NEV

: Msg 147, Level 15, State 1, Line 6
An aggregate may not appear in the WHERE clause unless it is in a subquery contained in a HAVING clause or a select list, and the column being aggregated is an outer reference.

Total execution time: 00:00:00.002